In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.shape

(569, 33)

In [4]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [5]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### train test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

### scaling

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
X_train

array([[-0.18677774, -0.24348593, -0.21751178, ..., -0.54482345,
        -0.36464549, -0.99456467],
       [-0.36940401, -0.44179041, -0.36175196, ..., -0.48680206,
        -1.52069842, -0.7519226 ],
       [-0.3609751 , -0.85020319, -0.36825292, ..., -0.26270388,
         0.1848985 , -0.34805835],
       ...,
       [-0.08563088,  1.04313238, -0.01598184, ...,  1.2966397 ,
         2.57407455,  1.30082933],
       [-1.3884586 , -1.29166672, -1.33080218, ..., -0.97041408,
         0.64731967,  0.02291442],
       [-0.66160604, -0.28125821, -0.6888318 , ..., -0.65800282,
        -1.53745281, -0.82633283]])

In [9]:
y_train

93     B
397    B
475    B
364    B
24     M
      ..
64     M
467    B
203    M
63     B
297    M
Name: diagnosis, Length: 455, dtype: object

### Label Encoding

In [10]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [11]:
y_train

array([0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,

### Numpy arrays to PyTorch tensors

In [12]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [13]:
X_train_tensor.shape

torch.Size([455, 30])

In [14]:
y_train_tensor.shape

torch.Size([455])

### Defining the model

In [24]:
class MySimpleNN:
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
        self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)

    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred

    def loss_function(self, y_pred, y_train_tensor):
        # Clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

        # Calculate loss
        loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
        return loss


### Important Parameters

In [25]:
learning_rate = 0.1
epochs = 30

### Training Pipeline

In [28]:
# instantiate model
model = MySimpleNN(X_train_tensor)

# define loop
for epoch in range(1,epochs+1):
    # forward pass
    y_pred = model.forward(X_train_tensor)

    # loss calculate
    loss = model.loss_function(y_pred, y_train_tensor)

    # backward pass
    loss.backward()

    # parameters update
    with torch.no_grad():
        model.weights.data -= learning_rate * model.weights.grad
        model.bias.data    -= learning_rate * model.bias.grad

    # zero gradients - to prevent Gradient Accumulation
    model.weights.grad.zero_()
    model.bias.grad.zero_()

    # print loss in each epoch
    print(f'Epoch: {epoch}, Loss: {loss.item()}')

Epoch: 1, Loss: 3.0866781635935396
Epoch: 2, Loss: 2.9602936382472445
Epoch: 3, Loss: 2.8314907352937317
Epoch: 4, Loss: 2.6955786293623687
Epoch: 5, Loss: 2.55968345019814
Epoch: 6, Loss: 2.422190442891824
Epoch: 7, Loss: 2.282116605722882
Epoch: 8, Loss: 2.1421017261355275
Epoch: 9, Loss: 2.004885441146458
Epoch: 10, Loss: 1.8722780426578292
Epoch: 11, Loss: 1.7425677020340946
Epoch: 12, Loss: 1.618060406263275
Epoch: 13, Loss: 1.5030962398902095
Epoch: 14, Loss: 1.395117671175972
Epoch: 15, Loss: 1.2982809858729845
Epoch: 16, Loss: 1.2096240208688724
Epoch: 17, Loss: 1.1320990273404774
Epoch: 18, Loss: 1.0671480532985833
Epoch: 19, Loss: 1.0137903368963546
Epoch: 20, Loss: 0.9706093663979716
Epoch: 21, Loss: 0.935937308161005
Epoch: 22, Loss: 0.9080748494179729
Epoch: 23, Loss: 0.8854765494511599
Epoch: 24, Loss: 0.8668560003720804
Epoch: 25, Loss: 0.8512105987235054
Epoch: 26, Loss: 0.8377942565268025
Epoch: 27, Loss: 0.8260685870699079
Epoch: 28, Loss: 0.8156523125092808
Epoch: 29

In [29]:
model.bias

tensor([-0.1701], dtype=torch.float64, requires_grad=True)

### Evaluation

In [30]:
# model evaluation
with torch.no_grad():
    y_pred = model.forward(X_test_tensor)
    y_pred = (y_pred > 0.9).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.6608186960220337
